In [177]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from helita.sim import rh15d 
from helita.vis import rh15d_vis

from scipy.integrate import quadrature
from scipy.integrate import cumtrapz

from pathlib import Path

rhRepoPath = Path("/Users/elka127/Documents/code/rh")

# With the Bifrost Snapshot or with existing atmosperes' models

```
# There are four 1D models of flares on M-dwarfs (/mn/stornext/u3/matsc/rh/Atmos/mflares/). 
# These are:

# radyn_out.F13_dpl_GRID_pt1s.ncdf
# tx.m2F12-37.max2F12_Ec37_d3.ncdf
# tx.m5F12-37.max5F12_Ec37_d3.ncdf
# tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf

# the three "tx" models all have a low cut-off energy of 37 keV and a power index of 3 
# (see Allred et al. 2015) 
# but vary in maximum beam energy: 2e12, 5e12 and 1e13 erg/cm^2/s. 

#The model from Kowalski et al. 2015 is radyn_out.F13_dpl_GRID_pt1s.ncdf.
```



In [ ]:
import h5py
from mpl_toolkits.axes_grid1 import make_axes_locatable
dpath_sim = "/Users/elka127/Documents/uio/project-Mflare/mflares/"

In [132]:
# to read xarray.Dataset and see dimentions, coordinates, data variables and attributes 
import xarray
atmos = xarray.open_dataset(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.m5F12-37.max5F12_Ec37_d3.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf")
atmos

<xarray.Dataset>
Dimensions:               (nhydr: 6, nt: 1, nx: 52, ny: 1, nz: 249)
Dimensions without coordinates: nhydr, nt, nx, ny, nz
Data variables:
    temperature           (nt, nx, ny, nz) float32 ...
    velocity_z            (nt, nx, ny, nz) float32 ...
    electron_density      (nt, nx, ny, nz) float64 ...
    hydrogen_populations  (nt, nhydr, nx, ny, nz) float32 ...
    snapshot_number       (nt) int32 ...
    x                     (nx) float32 ...
    y                     (ny) float32 ...
    z                     (nt, nx, ny, nz) float64 ...
Attributes:
    description:     2.5324933e-71
    has_B:        0

# Plotting atmospheres

In [89]:
# to plot 2-d atmospheres with height scale as index 
fig, ax = plt.subplots()
#atmos.temperature.plot.hist()
atmos.temperature.plot()
#atmos.velocity_z.plot()
#atmos.electron_density.plot()

ax.set_xlabel('height')
ax.set_ylabel('time')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'time')

```
# additional way to see variables
```

In [ ]:
hf = h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
#hf = h5py.File(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf","r")
#hf = h5py.File(dpath_sim+"tx.m5F12-37.max5F12_Ec37_d3.ncdf","r")
#hf = h5py.File(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf","r")

In [ ]:
# To see what data is in this file, we can call the keys() method on the file object.
hf.keys() # variables in the Bifrost snapshot

# Plotting atmospheres in real height

In [133]:
atmosTemp = atmos.temperature 
#atmostemp

# print(atmosTemp[0,:,:,248])
# print()
# print(atmosTemp[0,:,:,247])

# 1 - nt(1) - ?
# 2 - nx(52) - время
# 3 - ny(1) - ?
# 4 - nz(249) - высота

# sm1 = atmosTemp[0,0,0,248]
# print(sm1)
# sm2 = atmosTemp[0,0,0]
# print(sm2)

#print(atmosTemp[0,:,:,:])

# ---

atmosHeight = atmos.z
atmosHeight

<xarray.DataArray 'z' (nt: 1, nx: 52, ny: 1, nz: 249)>
array([[[[10244341.371021, ...,  -137315.771231]],

        ...,

        [[10239592.10447 , ...,  -137315.771211]]]])
Dimensions without coordinates: nt, nx, ny, nz

In [135]:
fig, axs = plt.subplots(1,3,figsize=(15,5),facecolor='w', edgecolor='k')
#fig.subplots_adjust(hspace = 0.1,wspace=0.25,left=0.05,right=0.95,top=0.93,bottom=0.07)
#axs.ravel()

# im=axs[0].imshow(atmosTemp[0,:,:,248],cmap='plasma')
# im=axs[1].imshow(atmosTemp[0,:,:,247],cmap='plasma')
# im=axs[2].imshow(atmosTemp[0,:,:,246],cmap='plasma')

im=axs[0].plot(atmosTemp[0,:,:,247])
im=axs[1].plot(atmosHeight[0,:,:,247])
#im=axs[2].plot(atmosTemp[0,:,:,246])

# divider0 = make_axes_locatable(axs[0])
# cax0 = divider0.append_axes("right", size="5%", pad=0.05)
# cbar0=plt.colorbar(im,cax=cax0)
# cbar0.set_label('T (K)')

# divider1 = make_axes_locatable(axs[1])
# cax1 = divider1.append_axes("right", size="5%", pad=0.05)
# cbar1=plt.colorbar(im,cax=cax1)
# cbar1.set_label('T (K)')

#axs[0].set_xlabel('pixels')
axs[0].set_ylabel('pixels')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'pixels')

In [136]:
#H = hf['z'][0,:] 
# geometric height
plt.figure()
    plt.plot(
        atmos.height_scale[0,0]/1e6, 
        atmos.electron_density[0, 0, 0]
    )


IndentationError: unexpected indent (<ipython-input-136-df3dba25b9cd>, line 4)

In [ ]:
print(atmos.height_scale[0,0]/1e6)


# Atoms for CaII

```
# Nmetal
   10

# Metals
#  model file              ACTIVE/PASSIVE  INITIAL_SOLUTION   population file


# ../../Atoms/CaII.atom            PASSIVE    ZERO_RADIATION
  ../../Atoms/H_6.atom             PASSIVE    ZERO_RADIATION
  ../../Atoms/MgII-IRIS.atom       PASSIVE    ZERO_RADIATION
  ../../Atoms/CaII_PRD.atom        ACTIVE     ZERO_RADIATION
  ../../Atoms/Si.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/Al.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/Fe.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/He.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/N.atom               PASSIVE    LTE_POPULATIONS
  ../../Atoms/Na.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/S.atom               PASSIVE    LTE_POPULATIONS
```

In [178]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 20.0) & (wave < 4000.0)]

wave.sel(wavelength=500, method="nearest")
index500 = np.argmin(np.abs(wave.data - 500))

with open(rhRepoPath / "rh15d/run/ray.input", "w") as f:
    f.write("1.00\n")
    output = str(len(indices) + 1)
    for ind in indices:
        output += f" {ind}"
    output += f" {index500}\n"
    f.write(output)

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.


# Writing ray.input file

In [179]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 20.0) & (wave < 4000.0)]

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.


```
#if we want to save particular wavelenghts, 500 for example, and get its index
```

In [180]:
wave.sel(wavelength=500, method='nearest')

<xarray.DataArray 'wavelength' ()>
array(500.)
Coordinates:
    wavelength  float64 500.0
Attributes:
    units:    nm

In [181]:
index500 = np.argmin(np.abs(wave.data - 500))

```
# To save this into a file ray.input we do:
```

In [182]:
f = open('ray.input', 'w')  
# this will overwrite any existing file!
f.write('1.00\n')
output = str(len(indices) + 1)
for ind in indices:
    output += ' %i' % ind
output += ' %i\n' % index500 
f.write(output)
f.close()

# Create own wavetable file

```
# Writes RH wave file (in xdr format). All wavelengths should be in nm.

# Parameters
# ----------
# start: number
#     Starting wavelength.
# end: number
#     Ending wavelength (non-inclusive)
# step: number
#     Wavelength separation
# outfile: string
#     Name of file to write.
# ewave: 1-D array, optional
#    Array of existing wavelengths. Program will make discard points
#    to make sure no step is enforced using these points too.
# air: boolean, optional
#     If true, will at the end convert the wavelengths into vacuum
#     wavelengths.
```

In [162]:
def make_wave_file(mywave, start=20, end=4000, step=0.01, new_wave=None,
                   ewave=None, air=False):
  

SyntaxError: unexpected EOF while parsing (<ipython-input-162-79697deb18d6>, line 3)

In [ ]:
from helita.sim import rh15d
rr = rh15d.Rh15dout()
# this will write wavelenghts from xxx to xxx nm, x.xx nm spacing:
rh15d.make_wave_file('my.wave', 20, 4000, 0.1)
# this will write an existing array "my_waves", if it exists
#rh15d.make_wave_file('my.wave', ewave=my_waves)

# Intencity plot for Ca II

In [183]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.


In [184]:
data.ray.intensity.plot()
#plt.axis([20, 1000, 0, 62])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Calculating optical depths

In [172]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
print(data.ray.wavelength_selected)
print("-")
print(data.ray.wavelength_selected.data)

# first column
height = data.atmos.height_scale[0, 0].dropna('height')  
# index of 500 nm
index500 = np.argmin(np.abs(data.ray.wavelength_selected.data - 500))  

tau500 = cumtrapz(data.ray.chi[0, 0, :, index500].dropna('height'), x=-height)
# ensure tau500 and height have same size
tau500 = np.concatenate([[1e-20], tau500])  

 

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.
<xarray.DataArray 'wavelength_selected' (wavelength_selected: 44)>
array([392.926626, 393.079987, 393.190157, 393.26934 , 393.326292, 393.367295,
       393.396854, 393.418205, 393.433666, 393.444901, 393.453105, 393.459133,
       393.463602, 393.466951, 393.469496, 393.471466, 393.473022, 393.474282,
       393.475329, 393.476223, 393.477007, 393.477713, 393.478419, 393.479204,
       393.480098, 393.481145, 393.482405, 393.483961, 393.48593 , 393.488476,
       393.491825, 393.496293, 393.502322, 393.510526, 393.521761, 393.537222,
       393.558572, 393.588132, 393.629135, 393.686087, 393.76527 , 393.87544 ,
       394.028801, 500.      ])
Coordinates:
  * wavelength_selected  (wavelength_selected) float64 392.9 393.1 ... 500.0
Attributes:


In [173]:
fig, ax = plt.subplots()
ax.plot(height / 1e6, tau500)  # height in Mm
ax.set_xlabel('H (Mm)')
ax.set_ylabel(r'$\tau$$_{500}$')
ax.set_yscale('log')
ax.axhline(y=1,linestyle='dashed',color='black')
ax.axvline(x=0.5,linestyle='dashed',color='black')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Plot the departure coefficients for the levels of Ca II



In [185]:
# plt.figure()
# for i in range(5):
#     plt.plot(data.atmos.height_scale[0,0]/1e6, 
#              data.atom_CA.populations[i, 0, 0]/data.atom_CA.populations_LTE[i, 0, 0],
#              label='Level %i' % (i + 1))
# plt.legend(loc="upper left")
# plt.figure()
# for i in range(5):
#     plt.plot(data.atmos.height_scale[0,0]/1e6, 
#              data.atom_CA.populations[i, 0, 0]/data.atom_CA.populations_LTE[i, 0, 0],
#              label='Level %i' % (i + 1))
fig, ax = plt.subplots()
ax.plot(tau500,data.atom_CA.populations[1, 0, 0]/data.atom_CA.populations_LTE[1, 0, 0], label='Level %1' % (1)) 
plt.legend(loc="upper left")
plt.xlabel("tau500")
plt.ylim(0.05, 2)
plt.ylabel("Departure coefficients")
plt.ylim(0.05, 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: incomplete format

# от тау 500

In [186]:
# xVals = data.atmos.height_scale[0,0]/1e6
# with open("/tmp/x.txt", "w") as f:
#     for x in xVals.data:
#         f.write(f"{x}\n")

print(data.atom_CA.populations[0,22,0])
# yVals = data.atom_CA.populations[0,0,0]
# with open("/tmp/y.txt", "w") as f:
#     for y in yVals.data:
#         f.write(f"{y}\n")


#print(data.atom_CA.populations[0,0,0][247])
# print("- - - - -")
# print(data.atom_CA.populations[1,0,0])
# print("- - - - -")
# print(data.atom_CA.populations[2,0,0])

<xarray.DataArray 'populations' (height: 249)>
array([1.732042e+00, 1.721472e+00, 1.717624e+00, ..., 3.084783e+18,
       2.959077e+18, 2.871737e+18], dtype=float32)
Coordinates:
    x        float64 32.0
    y        float64 10.0
Dimensions without coordinates: height


In [187]:
import matplotlib.animation as animation

fig, ax = plt.subplots()

line, = ax.plot(
    data.atmos.height_scale[0,0]/1e6,
    data.atom_CA.populations[0,0,0]
)

def animate(i):
    #line.set_xdata(x)
    line.set_ydata(data.atom_CA.populations[0, i, 0][0])
    return line,

ani = animation.FuncAnimation(
    fig,
    animate,
    np.arange(52),
    #blit=True,
    repeat=False
)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [188]:
plt.figure()
for i in range(5):
    plt.plot(
        data.atmos.height_scale[0,0]/1e6, 
        data.atom_CA.populations[i, 51, 0]/data.atom_CA.populations_LTE[i, 51, 0],
        label='Level %i' % (i + 1)
    )
plt.legend(loc="upper left")
plt.xlabel("Height (Mm)")
plt.ylabel("Departure coefficients")
plt.xlim(0,11)
plt.ylim(-20,10000)
#/data.atom_CA.populations_LTE[i, 100, 0]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-20.0, 10000.0)

# Source function

In [175]:
rh15d_vis.SourceFunction(data);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='wavelength', max=43), Checkbox(value=True, description='…